## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

In [ ]:
!pip install keras --upgrade

In [ ]:
!pip install ultralytics roboflow

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./datasets/{file_name[41:-4]}/')

In [ ]:
dataset_extract('/content/drive/MyDrive/DL_aivle/Mini0411/cool_roof_images.zip')
dataset_extract('/content/drive/MyDrive/DL_aivle/Mini0411/cool_roof_yolo_labels.zip')

Extracting: 100%|██████████| 204/204 [00:00<00:00, 384.99files/s]


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="EsvOFdnlJn0m04xJo3kR")
project = rf.workspace("aivle-lcaqt").project("aivle-4")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.47, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to AIVLE-4차-1 in yolov8:: 100%|██████████| 306/306 [00:00<00:00, 7085.64it/s]


In [ ]:
import shutil
import os

# 원본 디렉토리
source_dir = "/content/AIVLE-4차-1/train/labels"

# 대상 디렉토리
target_dir = "/content/datasets/train/labels"

# 대상 디렉토리가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 원본 디렉토리의 파일들을 대상 디렉토리로 이동
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    target_file = os.path.join(target_dir, filename)
    shutil.move(source_file, target_file)

print("파일 이동이 완료되었습니다.")


파일 이동이 완료되었습니다.


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
from ultralytics import YOLO, settings

In [ ]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [ ]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [ ]:
import os
import random
import shutil

# 랜덤 시드 설정
random.seed(2024)

# 데이터 디렉토리 설정
data_dir = "/content/datasets/"
train_dir = "/content/datasets/train/"
val_dir = "/content/datasets/valid/"

# train과 val 데이터 디렉토리 생성
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

# train 데이터셋과 val 데이터셋을 분할할 비율 설정
train_ratio = 0.8

# 각 데이터셋의 이미지 파일과 레이블 파일 목록 가져오기
image_files = [f for f in os.listdir(os.path.join(data_dir, 'cool_roof_images'))]
label_files = [f for f in os.listdir(os.path.join(data_dir, 'cool_roof_yolo_labels/obj_train_data'))]

# train 데이터셋에 속할 이미지 파일과 레이블 파일 선택
num_train = int(len(image_files) * train_ratio)
random.shuffle(image_files)
train_image_files = image_files[:num_train]
val_image_files = image_files[num_train:]

# train 데이터셋과 val 데이터셋에 속한 이미지 파일과 레이블 파일 복사
for image_file in train_image_files:
    shutil.copy(os.path.join(data_dir, 'cool_roof_images', image_file), os.path.join(train_dir, 'images', image_file))
    label_file = image_file.replace('.jpg', '.txt')  # 이미지 파일에 대응하는 레이블 파일
    shutil.copy(os.path.join(data_dir, 'cool_roof_yolo_labels/obj_train_data', label_file), os.path.join(train_dir, 'labels', label_file))

for image_file in val_image_files:
    shutil.copy(os.path.join(data_dir, 'cool_roof_images', image_file), os.path.join(val_dir, 'images', image_file))
    label_file = image_file.replace('.jpg', '.txt')  # 이미지 파일에 대응하는 레이블 파일
    shutil.copy(os.path.join(data_dir, 'cool_roof_yolo_labels/obj_train_data', label_file), os.path.join(val_dir, 'labels', label_file))


In [ ]:
import cv2
import shutil
import random
import os

random.seed(2024)

# 데이터 폴더 및 증강된 데이터 저장 폴더 경로 설정
data_dir = "/content/datasets/"
image_folder = os.path.join(data_dir, "cool_roof_images")
label_folder = os.path.join(data_dir, "cool_roof_yolo_labels/obj_train_data")
train_image_folder = os.path.join(data_dir, "train/images/")
train_label_folder = os.path.join(data_dir, "train/labels/")
val_image_folder = os.path.join(data_dir, "valid/images/")
val_label_folder = os.path.join(data_dir, "valid/labels/")

# train 데이터셋과 val 데이터셋을 분할할 비율 설정
train_ratio = 0.8

# 이미지 폴더에 있는 각 이미지에 대해 증강을 수행하고 저장
for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    label_file = image_file.replace('.jpg', '.txt')
    label_path = os.path.join(label_folder, label_file)

    # 이미지 증강 및 저장
    image = cv2.imread(image_path)

    # 이미지 크기 저장
    height, width = image.shape[:2]

    # 가우시안 블러 적용
    blurred_image = cv2.GaussianBlur(image, (0, 0), 5)

    # 무작위로 train 또는 val에 이미지를 할당
    if random.random() < train_ratio:
        save_image_folder = train_image_folder
        save_label_folder = train_label_folder
    else:
        save_image_folder = val_image_folder
        save_label_folder = val_label_folder

    # 증강된 이미지 저장
    save_image_name = os.path.splitext(image_file)[0] + "_aug.jpg"
    save_image_path = os.path.join(save_image_folder, save_image_name)
    cv2.imwrite(save_image_path, blurred_image)

    # 증강된 레이블 파일 이름 생성
    save_label_name = os.path.splitext(label_file)[0] + "_aug.txt"
    save_label_path = os.path.join(save_label_folder, save_label_name)

    # 레이블 파일 복사 및 저장
    shutil.copy(label_path, save_label_path)


### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
import yaml

In [ ]:
with open('/content/data.yaml', 'r+') as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    # display(film)
    film['train'] = './datasets/train/images'
    film['val'] = './datasets/valid/images/'
    yaml.dump(film, f)

In [ ]:
file_count = 0

for filename in os.listdir('/content/datasets/train/images'):
    if os.path.isfile(os.path.join('/content/datasets/train/images', filename)):
        file_count += 1

print("폴더 내 파일의 수:", file_count)

폴더 내 파일의 수: 569


### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
import keras

In [ ]:
keras.backend.clear_session()

In [ ]:
model = YOLO(model='yolov8n.pt', task='detect')

In [ ]:
model.train(data='/content/data.yaml',
                    epochs=100,
                    patience=10,
                    imgsz=640,
                    seed=2024,
                    pretrained=False,
                    cls=0.6,
                    iou=0.6,
                    )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /content/datasets/train/labels.cache... 569 images, 0 backgrounds, 0 corrupt: 100%|██████████| 569/569 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 612, len(boxes) = 4981. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/datasets/valid/labels.cache... 81 images, 0 backgrounds, 0 corrupt: 100%|██████████| 81/81 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train6
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         81        594      0.956      0.965      0.982      0.839



100 epochs completed in 0.665 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.3MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.23it/s]


                   all         81        594      0.961      0.958       0.98      0.841
                CoolRF         81        102      0.948      0.941       0.97      0.848
                    RF         81        492      0.974      0.975       0.99      0.834
Speed: 0.3ms preprocess, 3.4ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bb888981510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model.val()

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/datasets/valid/labels.cache... 81 images, 0 backgrounds, 0 corrupt: 100%|██████████| 81/81 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.24s/it]


                   all         81        594      0.961      0.956       0.98      0.839
                CoolRF         81        102      0.948      0.941       0.97      0.844
                    RF         81        492      0.974      0.972       0.99      0.834
Speed: 2.0ms preprocess, 10.5ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to runs/detect/train63


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bb8192ca230>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
image_path = '/content/test/*'

In [ ]:
model.predict(image_path, save=True)


image 1/5 /content/test/1.png: 544x640 11 CoolRFs, 3 RFs, 20.9ms
image 2/5 /content/test/2.png: 640x640 12 RFs, 12.2ms
image 3/5 /content/test/3.png: 608x640 1 CoolRF, 5 RFs, 13.0ms
image 4/5 /content/test/4.png: 640x640 1 CoolRF, 22 RFs, 10.5ms
image 5/5 /content/test/5.png: 608x640 1 CoolRF, 6 RFs, 11.3ms
Speed: 3.2ms preprocess, 13.6ms inference, 1.7ms postprocess per image at shape (1, 3, 608, 640)
Results saved to runs/detect/train64


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'CoolRF', 1: 'RF'}
 obb: None
 orig_img: array([[[ 66,  37,  36],
         [ 60,  29,  30],
         [ 62,  30,  32],
         ...,
         [120, 126, 121],
         [121, 127, 123],
         [120, 126, 122]],
 
        [[ 65,  36,  36],
         [ 62,  31,  32],
         [ 62,  31,  32],
         ...,
         [129, 134, 130],
         [126, 131, 129],
         [122, 128, 126]],
 
        [[ 69,  40,  40],
         [ 65,  34,  35],
         [ 61,  30,  31],
         ...,
         [130, 136, 132],
         [125, 131, 128],
         [120, 126, 124]],
 
        ...,
 
        [[110,  99, 105],
         [109,  98, 106],
         [108,  98, 106],
         ...,
         [112, 104, 114],
         [116, 107, 117],
         [118, 110, 120]],
 
        [[110,  99, 105],
         [109,  98, 106],
         [109,  99, 107],
         ...,
         

In [57]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

image_directory = '/content/runs/detect/train64'

for filename in os.listdir(image_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_directory, filename)
        image = mpimg.imread(image_path)
        plt.imshow(image)
        plt.axis('off')